# Chianson Siu
This script reads in a Washington census shapefile and for each county in Washington, creates a GeoJSON file whose filename is the county name and whose contents are the polygons for the block groups within that county. Additionally, the population for each county in 2017 is analyzed and the top ten highest populated counties are printed in descending order.
## Timeline
This script was finished on 2/2/2018 and took roughly 10 hours to write.

In [76]:
# import system paths and append directories to the ArcGIS loaction on local computer
import sys
import os
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\ArcToolbox\\Scripts')

# import arcpy package and set the working directory
import arcpy
workspace = "C:\\Users\\cjms2\\geog458\\lab_1\\urban_rural_ass\\"
arcpy.env.workspace = workspace

# set environment for Geo4W64 to create geoJson later
from subprocess import call
os.environ["GDAL_DATA"] = "C:\\OSGeo4W64\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W64\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W64\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W64\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W64\\apps\\msys\\bin;C:\\OSGeo4W64\\apps\\Python27\\Scripts"


## 1. Creating GeoJson Files for each County in Washington

In [77]:
washTable = "WashingtonFIPS.dbf" # Washington county codes table
censusShapeF = "saep_bg10\\saep_bg10.shp" # Washington census table

In [67]:
# sptial many-to-one table join. Census table is the destination and washington codes table is the input
# the tables are joined on the "FIPSCounty" attribute, adding a "CountyName" attribute to the
# destination table
arcpy.JoinField_management(censusShapeF,"COUNTYFP10",washTable,"FIPSCounty")

<Result 'C:\\Users\\cjms2\\geog458\\lab_1\\urban_rural_ass\\saep_bg10\\saep_bg10.shp'>

In [78]:
allCountyNames = [] # create empty list to hold all county names as strings
cursor = arcpy.da.SearchCursor(washTable, "CountyName") # search the WA codes table

# create a list based on values in the CountyName column of the washington codes table
for row in cursor:
    allCountyNames.append(str(row[0])) # append all county names to the list holding all counties as strings
del cursor

In [80]:
# get the spatial reference system of the census shapefile
censusSr = arcpy.Describe(censusShapeF).spatialReference

# get the geometry type of the census shapefile
censusGeometry = arcpy.Describe(censusShapeF).shapeType

# setting optional parameters of the create feature class management function using
# the original census shapefile as a template
geometry_type = censusGeometry
template = censusShapeF
has_m = "SAME_AS_TEMPLATE"
has_z = "SAME_AS_TEMPLATE"
spatial_reference = censusSr

In [81]:
# intialize list that will hold all insert cursors for each county
# insert cursors created in advance to save computation time
# when populating future shapefiles
insertCursors = []

# for each county, create a folder subdirectory with the county as the folder name.
# Then create a new shape file within each subdirectory named after the county
for countyName in allCountyNames: 
    dirResult = workspace + "result\\" + countyName # paths for each county folder
    dirShape = dirResult + "\\" + countyName + ".shp" # paths for each county shapefile
    
    if not os.path.exists (dirShape): # check if shapefile exists
        os.makedirs(dirResult) # make new path (folder) for each county if it does not exist
        # create a new feature class for each county using the census shape file as a template
        arcpy.CreateFeatureclass_management(dirResult,
                                            countyName + ".shp",
                                            geometry_type,
                                            template,
                                            has_m,
                                            has_z,
                                            spatial_reference)     
    # create and store new insert cursor for each county
    insertCursors.append(arcpy.da.InsertCursor(dirShape, ["SHAPE@","*"])) 


In [82]:
# get the fields of the census table to find index of the CountyName attribute
censusFields = [f.name for f in arcpy.ListFields(censusShapeF)]

# get index of the CountyName attribute in census table
countyIndex = censusFields.index("CountyName")

# cursor to search through the census table
countySearch = arcpy.da.SearchCursor(censusShapeF,["Shape@","*"])

# inserts each row in the census table into their corresponding county shapefile
for row in countySearch:
    # searches for the index of county names in county list and uses it
    # to reference the correct insert cursor stored in the insertCursors list
    insertCursors[allCountyNames.index(row[countyIndex + 1])].insertRow(row)
del countySearch
del insertCursors[:] # deletes all insert cursors

In [84]:
# get the spatial reference system EPSG code for the census shapefile
censusSrCode = censusSr.factoryCode

# create and set a folder for the GeoJson files if it does not already exist
geoJsonDir = workspace + "geo_json_files"
if not os.path.exists(geoJsonDir): # check if geo_json_files folder already exists
    os.makedirs(geoJsonDir) # if not, create one

# access each county's shapefile and convert it to a geojson file and store in the geo_json_files folder
# each geojson file is named after the county it holds data for.
for countyName in allCountyNames:
    call(["C:\\OSGeo4W64\\bin\\ogr2ogr",
      "-f","GeoJSON","-t_srs","WGS84",
      "-s_srs","EPSG:" + str(censusSrCode),
      workspace + "geo_json_files\\" + countyName + ".geojson",
      workspace + "result\\" + countyName + "\\" + countyName + ".shp"])

## 2. Top 10 Counties by Population

In [85]:
# intialize dictionary with counties as the key and 0 population as the value
pop2017 = {}
pop2017 = pop2017.fromkeys(allCountyNames, 0)

# search through census table to sum populations for each county
cursor = arcpy.da.SearchCursor(censusShapeF,["POP2017","CountyName"])
for row in cursor:
    pop2017[row[1]] += row[0] # add population data to corresponding county in dictionary entry
del cursor

# sort dictionary of counties by population values and place in descending order
pop2017Top10 = sorted(pop2017, key=pop2017.get, reverse=True)[0:10] # reverse=True to get desceding order
print("2017 Top 10 counties by population")

# print numbered list of the top 10 counties and their respective populaiton
for i in range(0, 10):
    print (str(i + 1) + ". " + pop2017Top10[i] + ": " + str(pop2017[pop2017Top10[i]]))

2017 Top 10 counties by population
1. King: 2153700.0
2. Pierce: 859400.0
3. Snohomish: 789400.0
4. Spokane: 499800.0
5. Clark: 471000.0
6. Thurston: 276900.0
7. Kitsap: 264300.0
8. Yakima: 253000.0
9. Whatcom: 216300.0
10. Benton: 193500.0
